**1. Importing all libraries**

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import sqlite3 

**2. Connecting dataset**

In [25]:
def create_connection(db_file):
    global conn
    if 'conn' in globals() and conn:
        conn.close()
    conn = sqlite3.connect(db_file)
    return conn

conn = create_connection('mental_health.sqlite')

**3. Checking database**

In [33]:
query = """
SELECT name 
FROM sqlite_master 
WHERE type='table';
"""
tables = conn.execute(query).fetchall()
tables = [row[0] for row in tables] 

print("Tables in the database:")
for table in tables:
    print(f" - {table}")


Tables in the database:
 - Answer
 - Question
 - Survey


By checking table names from base table schema information I can tell there are 3 tables

In [32]:
table_query = """
SELECT name 
FROM sqlite_master 
WHERE type='table';
"""
tables = conn.execute(table_query).fetchall()

for table in tables:
    table_name = table[0]  

    row_count_query = f"""
    SELECT COUNT(*) 
    FROM {table_name};
    """
    row_count = conn.execute(row_count_query).fetchone()[0]

    column_query = f"""
    PRAGMA table_info({table_name});
    """
    columns = conn.execute(column_query).fetchall()
    column_names = [col[1] for col in columns]

    print(f"\nTable: {table_name}")
    print(f" - Rows: {row_count}")
    print(f" - Columns: {', '.join(column_names)}")



Table: Answer
 - Rows: 236898
 - Columns: AnswerText, SurveyID, UserID, QuestionID

Table: Question
 - Rows: 105
 - Columns: questiontext, questionid

Table: Survey
 - Rows: 5
 - Columns: SurveyID, Description


Checking each tables row count and columns with their name tags. From this I can tell that Answers is the main table which connects both Survey and Question tables.
There are 5 surveys, 105 questions and 236898 entries consisting answers. 

**4. Checking NULL values**

In [31]:
query = """
SELECT m.name AS table_name, p.name AS column_name 
FROM sqlite_master m
JOIN pragma_table_info(m.name) p
WHERE m.type='table';
"""
table_columns = conn.execute(query).fetchall() 

results = []
for table, column in table_columns:
    null_count = conn.execute(f"SELECT COUNT(*) FROM {table} WHERE {column} IS NULL;").fetchone()[0]
    if null_count > 0:
        results.append((table, column, null_count))

if results:
    print("\nNULL Values Found:")
    for table, column, count in results:
        print(f"Table: {table} | Column: {column} | NULLs: {count}")
else:
    print("No NULL values found in the database.")

No NULL values found in the database.


**5. Checking for duplicates**

In [34]:
query = """
SELECT QuestionID, UserID, COUNT(*) as duplicate_count
FROM Answer
GROUP BY QuestionID, UserID
HAVING COUNT(*) > 1;
"""

duplicates = conn.execute(query).fetchall()


if duplicates:
    print("\nDuplicate QuestionID per UserID found:")
    for row in duplicates:
        print(f"QuestionID: {row[0]} | UserID: {row[1]} | Duplicates: {row[2]}")
else:
    print("No duplicate QuestionID per UserID found.")


Duplicate QuestionID per UserID found:
QuestionID: 115 | UserID: 1262 | Duplicates: 2
QuestionID: 115 | UserID: 1264 | Duplicates: 2
QuestionID: 115 | UserID: 1265 | Duplicates: 2
QuestionID: 115 | UserID: 1266 | Duplicates: 3
QuestionID: 115 | UserID: 1268 | Duplicates: 2
QuestionID: 115 | UserID: 1270 | Duplicates: 2
QuestionID: 115 | UserID: 1275 | Duplicates: 2
QuestionID: 115 | UserID: 1277 | Duplicates: 4
QuestionID: 115 | UserID: 1279 | Duplicates: 2
QuestionID: 115 | UserID: 1280 | Duplicates: 2
QuestionID: 115 | UserID: 1290 | Duplicates: 2
QuestionID: 115 | UserID: 1292 | Duplicates: 4
QuestionID: 115 | UserID: 1295 | Duplicates: 2
QuestionID: 115 | UserID: 1301 | Duplicates: 2
QuestionID: 115 | UserID: 1304 | Duplicates: 2
QuestionID: 115 | UserID: 1306 | Duplicates: 3
QuestionID: 115 | UserID: 1307 | Duplicates: 4
QuestionID: 115 | UserID: 1310 | Duplicates: 2
QuestionID: 115 | UserID: 1314 | Duplicates: 2
QuestionID: 115 | UserID: 1327 | Duplicates: 2
QuestionID: 115 | Us

There are many duplicates anwers from many user to specific questions. To gain a better perspective, I will review the questions and answers to determine whether these duplicates are truly redundant or if they represent different answers to the same question.

In [35]:
query = """
SELECT * FROM Question
WHERE QuestionID IN (115, 116, 117);
"""
questions = conn.execute(query).fetchall()

print("\nQuestions with IDs 115, 116, and 117:")
for row in questions:
    print(row)


Questions with IDs 115, 116, and 117:
('If yes, what condition(s) have you been diagnosed with?', 115)
('If maybe, what condition(s) do you believe you have?', 116)
('Which of the following best describes your work position?', 117)


In [36]:
query = """
SELECT * FROM Answer
WHERE (QuestionID = 115 AND UserID = 1292)
   OR (QuestionID = 116 AND UserID = 2065)
   OR (QuestionID = 117 AND UserID = 1499);
"""

results = conn.execute(query).fetchall()

print("\nRecords for specified QuestionID & UserID pairs:")
for row in results:
    print(row) 


Records for specified QuestionID & UserID pairs:
('Anxiety Disorder (Generalized, Social, Phobia, etc)', 2016, 1292, 115)
('Mood Disorder (Depression, Bipolar Disorder, etc)', 2016, 1292, 115)
('Substance Use Disorder', 2016, 1292, 115)
('Addictive Disorder', 2016, 1292, 115)
('Anxiety Disorder (Generalized, Social, Phobia, etc)', 2016, 2065, 116)
('Mood Disorder (Depression, Bipolar Disorder, etc)', 2016, 2065, 116)
('Post-traumatic Stress Disorder', 2016, 2065, 116)
('Stress Response Syndromes', 2016, 2065, 116)
('Front-end Developer', 2016, 1499, 117)
('Back-end Developer', 2016, 1499, 117)
('Sales', 2016, 1499, 117)
('DevOps/SysAdmin', 2016, 1499, 117)


From this, I can tell that many users have provided more than one answer, indicating that these duplicates are not true duplicates but rather additional responses from users for a specific question. It would be incorrect to remove them.

**6. Question order** 

In [37]:
query = """
SELECT QuestionID
FROM Question 
ORDER BY QuestionID;
"""
question_ids = [row[0] for row in conn.execute(query).fetchall()]

missing_ids = [q for q in range(question_ids[0], question_ids[-1] + 1) if q not in question_ids]

if missing_ids:
    print("\nMissing QuestionIDs detected:")
    print(missing_ids)
else:
    print("All QuestionIDs are in proper sequence.")


Missing QuestionIDs detected:
[35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]


By observing the database I noticed that questions dont go in order. To not face problems in upcoming work, I check it and change the order. 

In [38]:
query = """
SELECT QuestionID
FROM Question
ORDER BY QuestionID;
"""
question_ids = [row[0] for row in conn.execute(query).fetchall()]

new_ids = list(range(1, len(question_ids) + 1))  
id_mapping = dict(zip(question_ids, new_ids)) 

for old_id, new_id in id_mapping.items():
    update_query = f"""
    UPDATE Question 
    SET QuestionID = {new_id} 
    WHERE QuestionID = {old_id};
    """
    conn.execute(update_query)

conn.commit()

In [39]:
query = """
SELECT QuestionID
FROM Question 
ORDER BY QuestionID;
"""
question_ids = [row[0] for row in conn.execute(query).fetchall()]

missing_ids = [q for q in range(question_ids[0], question_ids[-1] + 1) if q not in question_ids]


if missing_ids:
    print("\nMissing QuestionIDs detected:")
    print(missing_ids)
else:
    print("All QuestionIDs are in proper sequence.")

All QuestionIDs are in proper sequence.


**7. Standardizing the database**

In [40]:
for table in tables:
    column_query = f"PRAGMA table_info({table});"
    columns = conn.execute(column_query).fetchall()

    for col in columns:
        col_name = col[1]
        col_type = col[2]

        if "CHAR" in col_type or "TEXT" in col_type:
            update_query = f"""
            UPDATE {table}
            SET {col_name} = TRIM(LOWER({col_name}))
            WHERE {col_name} IS NOT NULL;
            """
            conn.execute(update_query)
            
        if "DATE" in col_type or "date" in col_name.lower():
            update_query = f"""
            UPDATE {table}
            SET {col_name} = strftime('%Y-%m-%d', {col_name})
            WHERE {col_name} IS NOT NULL;
            """
            conn.execute(update_query)

        if "INT" in col_type or "REAL" in col_type:
            update_query = f"""
            UPDATE {table}
            SET {col_name} = CAST({col_name} AS INTEGER)
            WHERE {col_name} NOT LIKE '%[^0-9]%';
            """
            conn.execute(update_query)

conn.commit()

 1.Trim whitespace and convert text to lowercase
 2.Fix date formatting to YYYY-MM-DD
 3.Ensure numerical columns have correct values


## **EDA**

Now I will examine the data to gain a comprehensive understanding of the respondents.

**1. Checking how many people reponed to each survey**

In [42]:
query = """
SELECT SurveyID, COUNT(DISTINCT UserID) AS respondent_count
FROM Answer
GROUP BY SurveyID
ORDER BY respondent_count DESC;
"""

survey_responses = conn.execute(query).fetchall()

print("\nSurvey Respondent Count:")
print("SurveyID | Respondent Count")
for survey_id, respondent_count in survey_responses:
    print(f"{survey_id:<8} | {respondent_count}")


Survey Respondent Count:
SurveyID | Respondent Count
2016     | 1433
2014     | 1260
2017     | 756
2018     | 417
2019     | 352


Here I can see that most of the answers comes from 2016 and 2014. There is no survey from 2015. 

In [57]:
query = """
WITH valid_ages AS (
    SELECT CAST(AnswerText AS INTEGER) AS age
    FROM Answer
    WHERE QuestionID = 1
    AND AnswerText GLOB '[0-9]*'
)
SELECT 
    (SELECT MIN(age) FROM valid_ages) AS youngest,
    (SELECT MAX(age) FROM valid_ages) AS oldest,
    (SELECT AVG(age) FROM valid_ages) AS mean_age,
    (SELECT age FROM valid_ages ORDER BY age LIMIT 1 OFFSET (SELECT COUNT(*)/2 FROM valid_ages)) AS median_age,
    (SELECT MAX(age) - MIN(age) FROM valid_ages) AS age_range
FROM valid_ages;
"""

age_stats = conn.execute(query).fetchone()
youngest, oldest, mean_age, median_age, age_range = age_stats

print("\nSQL-Based Age Statistics for All Surveys (QuestionID 1):")
print("-------------------------------------------------------------------")
print(f"Youngest Respondent: {youngest}")
print(f"Oldest Respondent: {oldest}")
print(f"Mean Age: {mean_age:.2f}")
print(f"Median Age: {median_age}")
print(f"Age Range: {age_range}")


SQL-Based Age Statistics for All Surveys (QuestionID 1):
-------------------------------------------------------------------
Youngest Respondent: 0
Oldest Respondent: 329
Mean Age: 33.97
Median Age: 33
Age Range: 329


As there are outlyers like 0 and 329 , I apply a range for this data extraction starting from 16 (some interns or young freelancers) and 80 (some seniors that work in IT sector)

In [59]:
query = """
WITH valid_ages AS (
    SELECT CAST(AnswerText AS INTEGER) AS age
    FROM Answer
    WHERE QuestionID = 1
    AND AnswerText GLOB '[0-9]*' 
    AND CAST(AnswerText AS INTEGER) BETWEEN 16 AND 80 
)
SELECT 
    (SELECT MIN(age) FROM valid_ages) AS youngest,
    (SELECT MAX(age) FROM valid_ages) AS oldest,
    (SELECT AVG(age) FROM valid_ages) AS mean_age,
    (SELECT age FROM valid_ages ORDER BY age LIMIT 1 OFFSET (SELECT COUNT(*)/2 FROM valid_ages)) AS median_age,
    (SELECT MAX(age) - MIN(age) FROM valid_ages) AS age_range
FROM valid_ages;
"""

age_stats = conn.execute(query).fetchone()
youngest, oldest, mean_age, median_age, age_range = age_stats

print("\nSQL-Based Age Statistics (Ages 16-80) for All Surveys (QuestionID 1):")
print("-------------------------------------------------------------------")
print(f"Youngest Respondent: {youngest}")
print(f"Oldest Respondent: {oldest}")
print(f"Mean Age: {mean_age:.2f}")
print(f"Median Age: {median_age}")
print(f"Age Range: {age_range}")


SQL-Based Age Statistics (Ages 16-80) for All Surveys (QuestionID 1):
-------------------------------------------------------------------
Youngest Respondent: 17
Oldest Respondent: 74
Mean Age: 33.86
Median Age: 33
Age Range: 57
